# Notebook que ejemplifica el proceso de selección de hiperparámetros

Se recomienda ver la notebook de ejemplo ```ejemplo_ajuste_m2.ipynb``` y el docuemnto **pdf** antes de examinar esta notebook.

## Importación de paquetes necesarios, funciones y parámetros necesarios

In [1]:
import os
from concurrent.futures import ProcessPoolExecutor
import pandas as pd

In [2]:
from uf3.data import io
from uf3.data import composition

c:\Users\RetailAdmin\anaconda3\envs\tesina\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from src.utils import run_selec_hyper, get_r_max_dict
from src.utils import element_list, r_min_map, r_min_obs
from src.utils import DATA_IN_PATH

## Definición de parámetros

In [4]:
# Seleccionar cantidad de vecinos a tomar
n_vecinos = 2 # Valores permitidos: 2, 3 y [2,3]. Con el valor 2 se toman valores a primer vecino igualmente para el par Ba-Ti
# Parámetros gráficas
zoom = False # Graficar con zoom o no
plot = True # Plotear y guardar gráficas en cada punto de la grilla
# Características del sistema: elementos y orden de la interacción (pares o trios)
degree = 2
chemical_system = composition.ChemicalSystem(element_list=element_list, degree=degree)
# Seteo de distancia de corte, cuando n_vecinos = 2 la interacción Ba-Ti es igualmente a primer vecino
r_max_map_dict = get_r_max_dict(n_vecinos) 
# Cantidad de splines a recortar en r_min y r_max, tienen que ver con la suavidad del potencial en esos valores de r
trailing_trim = 3
leading_trim = 0

In [5]:
# Parámetros asociados a los recursos destinados al ajuste en paralelo
n_cores = 4
n_batches = n_cores * 16  # Granularidad añadida para más actualizaciones de la barra de progreso
client = ProcessPoolExecutor(max_workers=n_cores) # Cliente

In [6]:
# Creación estructura para calculo de dispersión de fonones
from ase import Atoms, Atom
a = 4.012
d = a/2
estruct = Atoms([Atom('Ba',(0,0,0)), Atom('Ti',(d,d,d)), Atom('O',(d,d,0)), Atom('O',(0,d,d)),Atom('O',(d,0,d))],
                cell=[a,a,a],pbc=True)

## Lectura de datos

In [7]:
data_filename = DATA_IN_PATH / "Training_set.xyz"
with open(DATA_IN_PATH / "training_idx_70%.txt", "r") as f:
    training_idx = [int(idx) for idx in f.read().splitlines()] 

In [8]:
# Creación de un DataFrame con las características de las estructuras
data_coordinator = io.DataCoordinator()
data_coordinator.dataframe_from_trajectory(str(data_filename),
                                           prefix='dft')
energy_key = data_coordinator.energy_key # Parámetro usado en el ajuste: nombre de la columna que tiene la E en el DF
df_data = data_coordinator.consolidate()

## División de datos en conjuntos de entrenamiento y test

In [9]:
# Seteo de índices del DataFrame que corresponden a los conjuntos de train y test
training_keys = df_data.index[training_idx] # Índices de las estructuras del conjunto train
holdout_keys = df_data.index.difference(training_keys) # Índices de las estructuras del conjunto test

## Construcción de grilla de hiperparámetros

Esta grilla es 'de juguete'. Una grilla con más puntos puede consumir muchos recursos computacionales.

In [10]:
n_splines_list = list(range(24,26)) # Números de splines a testear
rango_reg_1 = [10**i for i in range(-6,-5)] #lambda_0
rango_reg_2 = [10**i for i in range(-10,-5)] #lambda_2
pesos_fuerza_energia = [0.2,0.3] # kappa

## Selección de hiperparámetros

Se obtiene un diccionario con las métricas alcanzadas por los distintos conjuntos de potenciales (información que también se guarda en archivos ```.csv```).

In [11]:
# Ejecutar selección hiperparámetros
dict_df_final = run_selec_hyper(chemical_system,n_splines_list,rango_reg_1,rango_reg_2,pesos_fuerza_energia,r_min_map,
                    r_max_map_dict,r_min_obs,df_data,client,energy_key,n_batches,training_keys,holdout_keys,
                    estruct,leading_trim,trailing_trim,plot,zoom)

100%|██████████| 64/64 [00:51<00:00,  1.25it/s]


('O', 'O') Correction: adjusted 12 coefficients.
('O', 'Ti') Correction: adjusted 7 coefficients.
('O', 'Ba') Correction: adjusted 10 coefficients.
('Ba', 'Ba') Correction: adjusted 15 coefficients.
('Ti', 'Ba') Correction: adjusted 15 coefficients.
('Ti', 'Ti') Correction: adjusted 16 coefficients.
('O', 'O') Correction: adjusted 12 coefficients.
('O', 'Ti') Correction: adjusted 7 coefficients.
('O', 'Ba') Correction: adjusted 10 coefficients.
('Ba', 'Ba') Correction: adjusted 15 coefficients.
('Ti', 'Ba') Correction: adjusted 15 coefficients.
('Ti', 'Ti') Correction: adjusted 16 coefficients.
('O', 'O') Correction: adjusted 12 coefficients.
('O', 'Ti') Correction: adjusted 8 coefficients.
('O', 'Ba') Correction: adjusted 10 coefficients.
('Ba', 'Ba') Correction: adjusted 15 coefficients.
('Ti', 'Ba') Correction: adjusted 16 coefficients.
('Ti', 'Ti') Correction: adjusted 16 coefficients.
('O', 'O') Correction: adjusted 12 coefficients.
('O', 'Ti') Correction: adjusted 8 coefficients.

100%|██████████| 64/64 [00:52<00:00,  1.22it/s]


('O', 'O') Correction: adjusted 13 coefficients.
('O', 'Ti') Correction: adjusted 8 coefficients.
('O', 'Ba') Correction: adjusted 11 coefficients.
('Ba', 'Ba') Correction: adjusted 16 coefficients.
('Ti', 'Ba') Correction: adjusted 16 coefficients.
('Ti', 'Ti') Correction: adjusted 16 coefficients.
('O', 'O') Correction: adjusted 13 coefficients.
('O', 'Ti') Correction: adjusted 8 coefficients.
('O', 'Ba') Correction: adjusted 11 coefficients.
('Ba', 'Ba') Correction: adjusted 16 coefficients.
('Ti', 'Ba') Correction: adjusted 16 coefficients.
('Ti', 'Ti') Correction: adjusted 16 coefficients.
('O', 'O') Correction: adjusted 13 coefficients.
('O', 'Ti') Correction: adjusted 8 coefficients.
('O', 'Ba') Correction: adjusted 11 coefficients.
('Ba', 'Ba') Correction: adjusted 16 coefficients.
('Ti', 'Ba') Correction: adjusted 16 coefficients.
('Ti', 'Ti') Correction: adjusted 16 coefficients.
('O', 'O') Correction: adjusted 13 coefficients.
('O', 'Ti') Correction: adjusted 8 coefficients.

### Conjuntos con mejores métricas

In [12]:
# Energía
pd.DataFrame(dict_df_final).sort_values('RMSE_E (meV/atom)')

,N_SPLINES,REGULARIZADOR_1B,REGULARIZADOR_2B,RMSE_E (meV/atom),RMSE_F (meV/angstrom),kappa,cantidad_vecinos
9,24,0.000001,1.000000e-06,2.568590,183.804027,0.3,2
1,24,0.000001,1.000000e-10,2.568680,183.447125,0.3,2
3,24,0.000001,1.000000e-09,2.568680,183.447123,0.3,2
5,24,0.000001,1.000000e-08,2.568680,183.447121,0.3,2
7,24,0.000001,1.000000e-07,2.568687,183.446961,0.3,2
11,25,0.000001,1.000000e-10,2.574617,185.312640,0.3,2
13,25,0.000001,1.000000e-09,2.574792,185.306498,0.3,2
15,25,0.000001,1.000000e-08,2.586138,185.256791,0.3,2
19,25,0.000001,1.000000e-06,2.598473,185.615922,0.3,2
17,25,0.000001,1.000000e-07,2.599096,185.469932,0.3,2


In [13]:
# Fuerza
pd.DataFrame(dict_df_final).sort_values('RMSE_F (meV/angstrom)')

,N_SPLINES,REGULARIZADOR_1B,REGULARIZADOR_2B,RMSE_E (meV/atom),RMSE_F (meV/angstrom),kappa,cantidad_vecinos
2,24,0.000001,1.000000e-09,3.187951,167.902059,0.2,2
4,24,0.000001,1.000000e-08,3.187951,167.902061,0.2,2
6,24,0.000001,1.000000e-07,3.187945,167.902206,0.2,2
8,24,0.000001,1.000000e-06,3.187975,168.154435,0.2,2
14,25,0.000001,1.000000e-08,3.253197,168.528814,0.2,2
16,25,0.000001,1.000000e-07,3.264986,168.561653,0.2,2
12,25,0.000001,1.000000e-09,3.240359,168.574000,0.2,2
0,24,0.000001,1.000000e-10,4.214691,168.640671,0.2,2
10,25,0.000001,1.000000e-10,4.305791,168.644365,0.2,2
18,25,0.000001,1.000000e-06,3.264803,168.654607,0.2,2


La función ```run_selec_hyper``` además genera una carpeta con información adicional (si el parámetro ```plot``` es igual a ```True```) para ayudar a decidir cuales conjuntos de potenciales son los "mejores": graficas de los potenciales, el RMSE y RMSF en cada estructura y gráficas de curvas de dispersión de fonones, al igual que se guardan archivos ```.csv``` con las métricas. Los potenciales seleccionados son los que se someten a pruebas mediante simulaciones de dinámica molecular.